<a href="https://colab.research.google.com/github/krixik-ai/krixik-docs/blob/main/docs/examples/single_module_pipelines/single_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json
from pathlib import Path

# preparación de demo - incuye instanciación de secretos, instalación de requerimientos, y definición de rutas
if os.getenv("COLAB_RELEASE_TAG"):
    # si estás usando este notebook en Google Colab, ingresa tus secretos acá
    MY_API_KEY = "TU_API_KEY_VA_AQUI"
    MY_API_URL = "TU_API_URL_VA_AQUI"

    # si estás usando este notebook en Google Colab, instala requerimientos y descarga los subdirectorios requeridos
    # instala el cliente Python de Krixik
    !pip install krixik

    # instala github-clone, que permite clonación fácil de los subdirectorios del repositorio de documentación https://github.com/krixik-ai/krixik-docs
    !pip install github-clone

    # clona los conjuntos de datos
    if not Path("data").is_dir():
        !ghclone https://github.com/krixik-ai/krixik-docs/tree/es-main/data
    else:
        print("ya se clonaron los conjuntos de datos de documentación!")

    # define la variable 'data_dir' para tus rutas
    data_dir = "./data/"

    # crea directorio de salidas
    from pathlib import Path

    Path(data_dir + "/salidas").mkdir(parents=True, exist_ok=True)

else:
    # si estás usando una descarga local de la documentación, define las rutas relativas a la estructura local de la documentación
    # importa utilidades
    sys.path.append("../../../")

    # define la variable 'data_dir' para tus rutas
    data_dir = "../../../data/"

    # si estás usando este notebook localmente desde el repositorio de documentación Krixik, carga tus secretos de un archivo .env ubicado en la base del repositorio de documentación
    from dotenv import load_dotenv

    load_dotenv("../../../.env")

    MY_API_KEY = os.getenv("MY_API_KEY")
    MY_API_URL = os.getenv("MY_API_URL")


# importa Krixik e inicializa sesión con tus secretos personales
from krixik import krixik

krixik.init(api_key=MY_API_KEY, api_url=MY_API_URL)

SUCCESS: You are now authenticated.


## *Pipeline* de Módulo Único: `summarize` (Resumen)
[🇺🇸 English version of this document](https://krixik-docs.readthedocs.io/latest/examples/single_module_pipelines/single_summarize/)

Este documento presenta una guía de cómo ensamblar y consumir un *pipeline* de módulo único que solo incluye un módulo [`summarize` (resumen)](../../modulos/modulos_ia/modulo_summarize_resumen.md). Se divide en las siguientes secciones:

- [Monta tu *Pipeline*](#monta-tu-pipeline)
- [Formato de Entrada Requerido](#formato-de-entrada-requerido)
- [Cómo Usar el Modelo Predeterminado](#como-usar-el-modelo-predeterminado)
- [Cómo Usar un Modelo No-Predeterminado](#como-usar-un-modelo-no-predeterminado)
- [Resumen Recursivo](#resumen-recursivo)

### Monta tu *Pipeline*

Primero crea un *pipeline* de módulo único con un módulo [`summarize` (resumen)](../../modulos/modulos_ia/modulo_summarize_resumen.md).

Usa el método [`create_pipeline`](../../sistema/creacion_de_pipelines/creacion_de_pipelines.md) para esto, incluyendo solamente una referencia de módulo [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) en el argumento `module_chain`.

In [2]:
# crea un pipeline con un solo módulo summarize
pipeline = krixik.create_pipeline(name="unico_summarize_1", module_chain=["summarize"])

### Formato de Entrada Requerido

El módulo [`summarize` (resumen)](../../modulos/modulos_ia/modulo_summarize_resumen.md) recibe como entradas documentos textuales con formato TXT, PDF, DOCX y PPTX, aunque estos últimos tres formatos son automáticamente convertidos a TXT al iniciar proceso.

Antes de procesar un archivo de entrada—uno válido para este *pipeline*—examínalo con el siguiente código:

In [3]:
# examina el contenido de un archivo de entrada válido
with open(data_dir + "input/1984_corto.txt", "r") as file:
    print(file.read())

It was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the
vile wind, slipped quickly through the glass doors of Victory Mansions,
though not quickly enough to prevent a swirl of gritty dust from entering
along with him.

The hallway smelt of boiled cabbage and old rag mats. At one end of it a
coloured poster, too large for indoor display, had been tacked to the wall.
It depicted simply an enormous face, more than a metre wide: the face of a
man of about forty-five, with a heavy black moustache and ruggedly handsome
features. Winston made for the stairs. It was no use trying the lift. Even
at the best of times it was seldom working, and at present the electric
current was cut off during daylight hours. It was part of the economy drive
in preparation for Hate Week. The flat was seven flights up, and Winston,
who was thirty-nine and had a varicose ulcer above his right ankle, went
slowly, resting se

### Como Usar el Modelo Predeterminado

Ahora procesa el archivo usando el modelo [predeterminado](../../modulos/modulos_ia/modulo_summarize_resumen.md#modelos-disponibles-en-el-modulo-summarize) del módulo [`summarize` (resumen)](../../modulos/modulos_ia/modulo_summarize_resumen.md): [`bart-large-cnn`](https://huggingface.co/facebook/bart-large-cnn).

Dado que este es el modelo predeterminado, no hace falta que especifiques qué modelo quieres usar a través del argumento opcional [`modules`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md#seleccion-de-modelo-por-medio-del-argumento-modules) del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md).

In [4]:
# procesa el archivo con el modelo predeterminado
process_output = pipeline.process(
    local_file_path=data_dir + "input/1984_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
)

La salida del proceso se reproduce con el siguiente código. Para aprender más sobre cada componente de esta salida, revisa la documentación del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md).

El archivo de salida se ha guardado en la ubicación indicada bajo `process_output_files`. El `file_id` del archivo procesado es el prefijo del nombre del archivo de salida en esta ubicación.

In [5]:
# nítidamente reproduce la salida de este proceso
print(json.dumps(process_output, indent=2))

{
  "status_code": 200,
  "pipeline": "single_summarize_1",
  "request_id": "21af82dc-c558-4d7d-b819-b97b91308994",
  "file_id": "718948f7-685a-4e8e-b610-254b454897ce",
  "message": "SUCCESS - output fetched for file_id 718948f7-685a-4e8e-b610-254b454897ce.Output saved to location(s) listed in process_output_files.",
  "warnings": [],
  "process_output": null,
  "process_output_files": [
    "../../../data/output/718948f7-685a-4e8e-b610-254b454897ce.txt"
  ]
}


Para confirmar que todo salió como esperabas, carga el archivo de texto de `process_output_files`:

In [6]:
# carga la salida del proceso del archivo
with open(process_output["process_output_files"][0], "r") as file:
    print(file.read())

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelt of boiled cabbage and old rag mats. At one end of
it it acoloured poster, too large for indoor display, had been tacked
to the wall. It depicted simply an enormous face, more than a
metre wide. Winston made for the stairs.

Inside the flat a fruity voice was reading out a list of
figures which had something to do with pig-iron. Winston turned a switch
and the voice sank somewhat, though the words were still distinguishable. He
moved over to the window: a smallish, frail figure, the meagreness of
his body merely emphasized by the blue overalls which were the uniform
of the party.

Winston kept his back turned to the telescreen. It was safer; though,
as he well knew, even a back can be revealing. A kilometre
away the Ministry of Truth, his place of work, towered vast and
white above the grimy landscape. Winston tried to squeeze out some childhood
memory that should tell him whether London had always been 

### Como Usar un Modelo No-Predeterminado

Para usar un modelo [no-predeterminado](../../modulos/modulos_ia/modulo_summarize_resumen.md#modelos-disponibles-en-el-modulo-summarize) como [`text-summarization`](https://huggingface.co/Falconsai/text_summarization), debes especificarlo a través del argumento [`modules`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md#seleccion-de-modelo-por-medio-del-argumento-modules) al usar el método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md):

In [7]:
# procesa el archivo con un modelo no-predeterminado
process_output = pipeline.process(
    local_file_path=data_dir + "input/1984_corto.txt",  # la ruta de archivo inicial en la que yace el archivo de entrada
    local_save_directory=data_dir + "output",  # el directorio local en el que se guardará el archivo de salida
    expire_time=60 * 30,  # data de este proceso se eliminará del sistema Krixik en 30 minutos
    wait_for_process=True,  # espera que el proceso termine antes de devolver control del IDE al usuario
    verbose=False,  # no mostrar actualizaciones de proceso al ejecutar el código
    modules={"summarize": {"model": "text-summarization"}},  # especifica un modelo no-predeterminado para este proceso
)

Puedes ver el nuevo resumen si cargas el archivo de salida con el siguiente código.

Los descuadres en puntuación son un artefacto del modelo, como indica la [tarjeta del modelo](https://huggingface.co/Falconsai/text_summarization) en [HuggingFace](https://huggingface.co/).

In [8]:
# carga la salida del proceso del archivo
with open(process_output_nd["process_output_files"][0], "r") as file:
    print(file.read())

Winston Smith, his chin nuzzled into his breast in an effort to
escape the vile wind, slipped quickly through the glass doors of Victory
Mansions . At one end of it a coloured poster, too large
for indoor display, had been tacked to the wall . It was
part of the economy drive in preparation for Hate Week . The
flat was seven flights up, and Winston went slowly, resting several times
on the way .

Winston turned a switch and the voice sank somewhat, though the words
were still distinguishable . The instrument (the telescreen, it was called) could
be dimmed, but there was no way of shutting it off completely
. He moved over to the window: a smallish, frail figure, the
meagreness of his body merely emphasized by the blue overalls which were
the uniform of the party . Outside, even through the shut window-pane,
the world looked cold .

There was no way of knowing whether you were being watched at
any given moment . How often, or on what system, the Thought
Police plugged in on any individu

### Resumen Recursivo

Si el resultado de resumir una vez no es lo suficientemente conciso, hay un elegante truco que puedes usar.

Una de las formas más prácticas para lograr resúmenes más cortos (tal vez más abstractos, pero igual representativos) es resumir recursivamente. En otras palabras, le alimentas el resumen antes creado al módulo `summarize` una vez más, así produciendo un resumen más breve. En esta sección aprenderás a hacer este proceso manualmente.

Para ingresar el <u>primer</u> resumen arriba generado en el módulo [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) debes repetir lo que hiciste anteriormente, pero con una diferencia: el archivo que alimentas como entrada es la salida resumida de ese primer proceso de resumen, y no el archivo original.

In [9]:
# asigna el resumen generado a una variable para encontrarlo facilmente
first_summary = process_output["process_output_files"][0]

# procesa este resumen a través del pipeline
process_output = pipeline.process(
    local_file_path=first_summary,  # alimenta al pipeline el resumen antes generado
    local_save_directory=data_dir + "output",
    expire_time=60 * 30,
    wait_for_process=True,
    verbose=False,
)

Una vez termina este proceso recibes un resumen aún más breve como archivo de salida.

Con el siguiente código puedes examinar este nuevo resumen:

In [10]:
# carga la salida del proceso del archivo
with open(process_output["process_output_files"][0], "r") as file:
    print(file.read())

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelled of boiled cabbage and old rag mats. At one end of
the hallway an acoloured poster, too large for indoor display, had been
tacked to the wall. It depicted simply an enormous face, more than
a metre wide.

Winston kept his back turned to the telescreen. It was safer; though,
he well knew, even a back can be revealing. A kilometre away
the Ministry of Truth, his place of work, towered vast and white.


Este es un resumen más conciso, y por ende un poco más abstracto, del texto original.

Ahora haz este proceso recursivo una vez más, buscando un resumen aún más breve del texto original.

El uso del método [`process`](../../sistema/parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md) es casi igual. La única diferencia es que `local_file_path`, que le dice a Krixik qué archivo usar como entrada, indica la salida del segundo resumen que generaste.

In [11]:
# asigna el segundo resumen generado a una variable para encontrarlo facilmente
second_summary = process_output["process_output_files"][0]

# procesa este resumen a través del pipeline una vez más
process_output = pipeline.process(
    local_file_path=second_summary,  # alimenta al pipeline el segundo resumen antes generado
    local_save_directory=data_dir + "output",
    expire_time=60 * 30,
    wait_for_process=True,
    verbose=False,
)

Ahora puedes ver el nuevo y muy corto resumen:

In [12]:
# load in the recursed summary from file
with open(process_output["process_output_files"][0], "r") as file:
    print(file.read())

Winston Smith walked through the glass doors of Victory Mansions. The hallway
smelled of boiled cabbage and old rag mats. A kilometre away, his
place of work, the Ministry of Truth, towered vast and white.


Puedes ver que este es un resumen muy reducido del texto original—reducido pero aún representativo.

Puedes obtener el mismo resultado (de resumir recursivamente tres veces) con un pipeline que contiene tres módulos [`summarize`](../../modulos/modulos_ia/modulo_summarize_resumen.md) consecutivos. Haz [clic aquí](../../ejemplos/ejemplos_pipelines_multi_modulo_sin_busqueda/multi_resumen_recursivo.md) para detallar un ejemplo de un *pipeline* de resumen recursivo en el que hacemos justamente esto.

In [13]:
# elimina todos los datos procesados pertenecientes a este pipeline
krixik.reset_pipeline(pipeline)